<h1>Exploiting Genetic Algorithms for Optimisation </h1>
<br>

<h2> The problem</h2>
We need to make cylindrical containers with fixed volume 30 cubic cm so that, given the diameter you can figure out its height, minimizing the building material. 
This mathematically simple minimization problem could be solved with a genetic algorithm driven by the diameter (or radius) that must always be positive. The rest of the problem definition is up to you. 

In [2]:
# Install dependencies
import math
import numpy as np

In [7]:
#1. Define function to minimise
def height (d):
    r = d/2
    pi = math.pi
    #volume = pi * (r**2) * h
    h = 30/(pi*(r**2))
    return h

#2. Define Fitness
def fitness (d):
    answer = height(d)

    if answer == 30:
        return 9999999
    else:
        return abs(1/(answer-30))
    
#3. Define selection methods
def ranked_fitness (current_generation):
    ranked_solutions = []
    for el in current_generation:
        ranked_solutions.append(fitness(el[0],el[1], el))
    ranked_solutions.sort()
    ranked_solutions.reverse()
    return ranked_solutions

#4. Define reproduction method

    

In [ ]:
#3. Initial Hyperparameters
pop_size = 100
max_generations = 1000


#4. Start with generation zero
elements = []
for i in range(pop_size):
    elements.append((np.random.randint(10000), np.random.randint(10000)))

print(elements[:5])

In [ ]:
#5. New generation iterations
fittest = []
for i in range(max_generations):
    ranked_fit = ranked_fitness(elements)
    
    

In [ ]:
"""
Genetic Algorithm implementation of finding coefficients of a polynomial
3.0 - 4.3 * x + 5.9 * x ** 2 - 5.2 * x ** 3 + 1.0 * x ** 4 = 0
"""

#%% import modules
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# np.set_printoptions(linewidth=np.inf)


#%% define a function to return the polynomial
def poly(x):
    return 3.0 - 4.3 * x + 5.9 * x ** 2 - 5.2 * x ** 3 + 1.0 * x ** 4


def gen_poly(array, x):
    return (
        array[0]
        + array[1] * x
        + array[2] * x ** 2
        + array[3] * x ** 3
        + array[4] * x ** 4
    )


#%% define function to measure fitness
### takes an array as input and fills the last column with fitness values
### take care to always keep one column for fitness values in starting array
### output is an array sorted by fitness
def fitness(array):
    # set initial values
    initial = np.array(
        [
            [-5, 1447.0],
            [-4, 703.4],
            [-3, 290.4],
            [-2, 92.8],
            [-1, 19.4],
            [0, 3],
            [1, 0.4],
            [2, -7.6],
            [3, -16.2],
            [4, 3.4],
            [5, 104],
        ]
    )
    # score is the total square deviation from initial. lower is better
    for ind in array:
        fitness = 0
        for x, y in zip(initial[:, 0], initial[:, 1]):
            fitness += (
                (
                    ind[0]
                    + ind[1] * x
                    + ind[2] * x ** 2
                    + ind[3] * x ** 3
                    + ind[4] * x ** 4
                )
                - y
            ) ** 2
        ind[5] = fitness
    return array[array[:, 5].argsort()]


#%% function to give non linear rank to sorted pool
def nl_rank(array):
    copy = deepcopy(array)
    # normalization factor for selection pressure 3 and individuals 50
    norm = 290.3359045831912
    # setting ranks based on position in the pool
    for i, j in enumerate(copy):
        j[-1] = 50 * 1.06 ** (49 - i) / norm
    return copy


#%% define function to carry out stochastic universal sampling
### takes non linear ranked array as input
### output is list of selected individuals
def sel_ind(nl_array):
    copy = deepcopy(nl_array)
    # normalize ranks
    norm = sum(copy[:, -1])
    copy[:, -1] = copy[:, -1] / norm

    # map intervals on range [0, 1]
    prob_list = list(copy[:, -1])
    intervals = []
    start = 0
    for prob in prob_list:
        end = start + prob
        intervals.append((start, end))
        start = end

    # selecting 6 individuals from the intervals
    rng = np.random.default_rng()
    points = [rng.uniform(0, 1 / 5)]
    for i in range(4):
        points.append(points[-1] + 1 / 5)
    index, i = [], 0
    for point in points:
        for j in range(i, len(intervals)):
            if intervals[j][0] < point < intervals[j][1]:
                index.append(j)
                i = j
                break
    return index


#%% define function to carry out mating. only unique pairings are considered
### each mating gives 2 children
def crossover(array, individuals):
    rng = np.random.default_rng()
    progeny = np.empty((0, 6))
    for i in range(len(individuals) - 1):
        for j in range(i + 1, len(individuals)):
            mate1 = rng.uniform(-0.25, 1.25, [1, 5]).squeeze()
            mate2 = rng.uniform(-0.25, 1.25, [1, 5]).squeeze()
            mutation1 = rng.uniform(-0.025, 0.025, [1, 5]).squeeze()
            mutation2 = rng.uniform(-0.025, 0.025, [1, 5]).squeeze()
            baby1 = (
                array[i, :5] * mate1 + array[j, :5] * (1 - mate1) + mutation1 * mate1
            )
            baby1 = np.append(baby1, 0)
            progeny = np.append(progeny, [baby1], axis=0)
            baby2 = (
                array[j, :5] * mate2 + array[i, :5] * (1 - mate2) + mutation2 * mate2
            )
            baby2 = np.append(baby2, 0)
            progeny = np.append(progeny, [baby2], axis=0)
    return fitness(progeny)


#%% helper function to print arrays to log
def arr_print(arr, count):
    print(f"#loop_count = {count}")
    print(arr)


#%% main
if __name__ == "__main__":
    # create rng instance
    rng = np.random.default_rng()

    # create parent pool
    # parent pool has 5 columns for coefficients and one for fitness
    pool = rng.uniform(-50, 50, [50, 6])

    # measure fitness of each parent and sort in decreasing order of fitness
    pool = fitness(pool)
    starting_fitness = pool[0, 5]

    # plotting the original curve
    plt.ion()
    fig, ax = plt.subplots()
    x = np.linspace(-5, 5, 200)
    ax.plot(x, poly(x), "r", lw=1, label="Original")
    ax.set_xlabel("X axis")
    ax.set_ylabel("Y axis")
    ax.set_ylim(-1000, 1000)
    ax.set_title("Comparision of curves")
    ax.legend()

    loop_count = 1

    while True:
        # plotting
        if loop_count == 1:
            ax.plot(x, gen_poly(pool[0, :], x), lw=1, label=f"Iteration = {loop_count}")
            ax.legend()
            fig.canvas.draw()
            plt.pause(0.0001)
            # arr_print(pool, loop_count)
        elif loop_count % 100 == 0:
            ax.plot(x, gen_poly(pool[0, :], x), lw=0.25, ls="solid")
            fig.canvas.draw()
            plt.pause(0.0000001)
            # arr_print(pool, loop_count)
        # add termination condition
        elif pool[0, 5] < 0.005:
            ax.plot(
                x,
                gen_poly(pool[0, :], x),
                "k",
                lw=1.5,
                label=f"Iteration = {loop_count}",
            )
            ax.set_xlim(-2, 5)
            ax.set_ylim(-40, 75)
            ax.legend()
            fig.canvas.draw()
            plt.pause(0.001)
            # arr_print(pool, loop_count)
            break
        # rank parents based on non linear ranking
        ranked = nl_rank(pool)

        # select individuals
        individuals = sel_ind(ranked)

        # create progeny
        progeny = crossover(ranked, individuals)

        # remove 20 worst individuals from pool
        pool = np.delete(pool, np.s_[-20:], axis=0)

        # add progeny to the new pool
        pool = np.vstack((pool, progeny[:20, :]))

        # sort pool according to fitness
        pool = pool[pool[:, 5].argsort()]

        loop_count += 1

    print(starting_fitness, pool[0, 5])


In [9]:
from random import random, sample, choice
from math import floor
from tqdm import tqdm
from numpy import array, dot, mean
from numpy.linalg import pinv
from sys import exit


def generate_data():
    """
    We will generate data with a clear pattern.
    This ensures we have an idea of the desired result.
    This is only for demonstration purposes, real data is needed in practice.
    """
    coeff = [0.4, -0.3, 0.2, -0.1]
    x = [[random() for j in range(len(coeff))] for i in range(1000)]
    y = [dot(i, coeff) for i in x]
    return array(x), array(y)


def multiple_linear_regression(inputs, outputs):
    """
    Get the best expected outcome.
    This is expected to equal the coefficients in generate_data().
    """
    X, Y = array(inputs), array(outputs)
    X_t, Y_t = X.transpose(), Y.transpose()
    coeff = dot((pinv((dot(X_t, X)))), (dot(X_t, Y)))
    Y_p = dot(X, coeff)
    Y_mean = mean(Y)
    SST = array([(i - Y_mean) ** 2 for i in Y]).sum()
    SSR = array([(i - j) ** 2 for i, j in zip(Y, Y_p)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(Y))
    return {'COD': COD, 'coeff': coeff, 'error': av_error}


def check_termination_condition(best_individual):
    """
    Check if the current_best_individual is better of equal to the expected.
    """
    if ((best_individual['COD'] >= 99.0)
            or (generation_count == max_generations)):
        return True
    else:
        return False


def create_individual(individual_size):
    """
    Create an individual.
    """
    return [random() for i in range(individual_size)]


def create_population(individual_size, population_size):
    """
    Create an initial population.
    """
    return [create_individual(individual_size) for i in range(population_size)]


def get_fitness(individual, inputs):
    """
    Calculate the fitness of an individual.
    Return the Coefficient of Determination, average error and weight.
    We use the error to get the best individual.
    """
    predicted_outputs = dot(array(inputs), array(individual))
    output_mean = mean(outputs)
    SST = array(
        [(i - output_mean) ** 2 for i in outputs]).sum()
    SSR = array(
        [(i - j) ** 2 for i, j in zip(outputs, predicted_outputs)]).sum()
    COD = (1 - (SSR / SST)) * 100.0
    av_error = (SSR / len(outputs))
    return {'COD': COD, 'error': av_error, 'coeff': individual}


def evaluate_population(population):
    """
    Evaluate a population of individuals and return the best among them.
    """
    fitness_list = [get_fitness(individual, inputs)
                    for individual in tqdm(population)]
    error_list = sorted(fitness_list, key=lambda i: i['error'])
    best_individuals = error_list[: selection_size]
    best_individuals_stash.append(best_individuals[0]['coeff'])
    print('Error: ', best_individuals[0]['error'],
          'COD: ', best_individuals[0]['COD'])
    return best_individuals


def crossover(parent_1, parent_2):
    """
    Return offspring given two parents.
    Unlike real scenarios, genes in the chromosomes aren't necessarily linked.
    """
    child = {}
    loci = [i for i in range(0, individual_size)]
    loci_1 = sample(loci, floor(0.5*(individual_size)))
    loci_2 = [i for i in loci if i not in loci_1]
    chromosome_1 = [[i, parent_1['coeff'][i]] for i in loci_1]
    chromosome_2 = [[i, parent_2['coeff'][i]] for i in loci_2]
    child.update({key: value for (key, value) in chromosome_1})
    child.update({key: value for (key, value) in chromosome_2})
    return [child[i] for i in loci]


def mutate(individual):
    """
    Mutate an individual.
    The gene transform decides whether we'll add or deduct a random value.
    """
    loci = [i for i in range(0, individual_size)]
    no_of_genes_mutated = floor(probability_of_gene_mutating*individual_size)
    loci_to_mutate = sample(loci, no_of_genes_mutated)
    for locus in loci_to_mutate:
        gene_transform = choice([-1, 1])
        change = gene_transform*random()
        individual[locus] = individual[locus] + change
    return individual


def get_new_generation(selected_individuals):
    """
    Given selected individuals, create a new population by mating them.
    Here we also apply variation operations like mutation and crossover.
    """
    parent_pairs = [sample(selected_individuals, 2)
                    for i in range(population_size)]
    offspring = [crossover(pair[0], pair[1]) for pair in parent_pairs]
    offspring_indices = [i for i in range(population_size)]
    offspring_to_mutate = sample(
        offspring_indices,
        floor(probability_of_individual_mutating*population_size)
    )
    mutated_offspring = [[i, mutate(offspring[i])]
                         for i in offspring_to_mutate]
    for child in mutated_offspring:
        offspring[child[0]] = child[1]
    return offspring

inputs, outputs = generate_data()
individual_size = len(inputs[0])
population_size = 1000
selection_size = floor(0.1*population_size)
max_generations = 50
probability_of_individual_mutating = 0.1
probability_of_gene_mutating = 0.25
best_possible = multiple_linear_regression(inputs, outputs)
best_individuals_stash = [create_individual(individual_size)]
initial_population = create_population(individual_size, 1000)
current_population = initial_population
termination = False
generation_count = 0
while termination is False:
    current_best_individual = get_fitness(best_individuals_stash[-1], inputs)
    print('Generation: ', generation_count)
    best_individuals = evaluate_population(current_population)
    current_population = get_new_generation(best_individuals)
    termination = check_termination_condition(current_best_individual)
    generation_count += 1
else:
    print(get_fitness(best_individuals_stash[-1], inputs))

Generation:  0


100%|██████████| 1000/1000 [00:00<00:00, 1439.90it/s]


Error:  0.03172091743311806 COD:  -28.33271628318752
Generation:  1


100%|██████████| 1000/1000 [00:00<00:00, 1445.27it/s]


Error:  0.016271198959333624 COD:  34.17191781926964
Generation:  2


...

Error:  0.0002478626374403543 COD:  98.99722681114433
Generation:  12


100%|██████████| 1000/1000 [00:00<00:00, 1432.78it/s]


Error:  0.00015546699482728326 COD:  99.37103011662143
Generation:  13


100%|██████████| 1000/1000 [00:00<00:00, 1434.79it/s]


Error:  0.00015546699482728326 COD:  99.37103011662143
{'COD': 99.37103011662143, 'error': 0.00015546699482728326, 'coeff': [0.3900647396485527, -0.2685295123111946, 0.18580058536812183, -0.09314616266543319]}
